<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/OpenAccess-GoogleScholar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open Access - Google Scholar
La base de datos oaudea.json que fue creada con: [oaudea.ipynb](./oaudea.ipynb)

Está basada en la combinación de:

* Web of Science con Scopus y Scielo (Web of science) para la extración de la columna de DOIS e ISSNs de las revistas  de los artículos de la UdeA
* oadoi.org para la identificación de artículos Open Access
* DOAJ para obetener la información del APC basada en la identificación de los ISSNs de las revistas
* Búsqueda en google scholar (GS) de cada artículo basada en DOI y en título

In [0]:
!pip install wosplus

In [0]:
import wosplus as wp

In [0]:
%%writefile drive.cfg
[FILES]
DOIS.xlsx               = 1bikNT7Gmp4G7dfeMuGsF-az7D8lskK0O
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj
oaudea.xlsx             = 1CcwobiEFACIbffNzNdLxpdxQukr8cZ5x
datos1.csv              = 11CyLRZZwVbgw6YAC-igRJ3mrkIwk0aaiXnd-EOofYTI
oa_doi.json             = 1pLaEtyM-aOhjsB6twr05OhyJToIM7h7e
oaudea.json             = 1BmqRoQDgfpOFjXBVEfI999uaN6XX6Fkd

Overwriting drive.cfg


In [0]:
oa=wp.wosplus('drive.cfg')

In [0]:
df=oa.read_drive_json('oaudea.json')

In [0]:
df.shape

(7990, 39)

In [0]:
df.loc[[1]]

,APC,APC_USD,DOI,GS_Journal,GS_PDF,GS_Year,GS_abstract,GS_authors,GS_cites,GS_cites_link,...,journal_issns,journal_name,language,oa_locations,published_date,publisher,title,updated,year,z_authors
1,0NOT,0,10.1016/s0370-2693(99)01040-0,Lorenzana,,-1,Skip to main content \n,A Perez,6,/scholar?cites=7204515635894942882&as_sdt=2005...,...,0370-2693,Physics Letters B,en,"[{'version': 'submittedVersion', 'host_type': ...",1999-10-01,Elsevier BV,An [SU(3)]4 supersymmetric grand unified model,2018-06-17T11:59:16.138320,1999,"[{'given': 'Abdel', 'family': 'Pérez-Lorenzana..."


Fix GS_cites

In [0]:
dfn=df[df.GS_cites==''].reset_index(drop=True)
dfy=df[df.GS_cites!=''].reset_index(drop=True)
dfn['GS_cites']=0
df=dfy.append(dfn).reset_index(drop=True)

## Análisis

Número de artículos Open Access

In [0]:
dfoa=df[df.Open_Access!=''].reset_index(drop=True)
dfc=df[df.Open_Access==''].reset_index(drop=True) # close 
dfoa.shape[0],dfc.shape[0]

(3628, 4362)

Gasto en OA

In [0]:
df.APC_USD.sum()

1001070

In [0]:
dfoa.APC_USD.sum()

1001070

Citas OA

In [0]:
dfoa.GS_cites.sum()

73963.0

Falta bajar para no OA ?

In [0]:
dfc.GS_cites.sum()

0